In [ ]:
import cv2
import numpy as np
import glob

# -------------------------------
# ฟังก์ชันสร้าง mask (ใช้ค่าสีชมพูใหม่และปรับปรุง mask)
# -------------------------------
def create_mask(image):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    # ใช้ค่าสีชมพูใหม่
    lower_pink = np.array([120, 20, 80])
    upper_pink = np.array([170, 100, 200])
    
    mask = cv2.inRange(hsv, lower_pink, upper_pink)
    
    # ปรับปรุง mask ด้วย median blur
    mask = cv2.medianBlur(mask, 5)
    
    # สร้าง kernel สำหรับ morphology
    kernel = np.ones((7, 7), np.uint8)
    
    # ปิดช่องว่างเล็กๆ
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    
    return mask

# -------------------------------
# โหลดภาพจริง
# -------------------------------
image = cv2.imread("../../../image//capture_1756180004.jpg")
if image is None:
    print("ERROR: Cannot load main image!")
    exit()

# ตัดส่วนบนออกเหมือนใน detect_object.ipynb
image = image[220:, :]

orig_mask = create_mask(image)

# Debug: แสดง mask ของภาพหลัก
print(f"Main image shape: {image.shape}")
print(f"Main mask shape: {orig_mask.shape}")
print(f"Main mask white pixels: {np.sum(orig_mask == 255)}")

cv2.imshow("Main Image", image)
cv2.imshow("Main Mask", orig_mask)
cv2.waitKey(1000)

# -------------------------------
# โหลด ROI template images และแปลงเป็น mask
# -------------------------------
roi_files = glob.glob("../../../image/template/template*.jpg")
print(f"Found template files: {roi_files}")

if len(roi_files) == 0:
    print("ERROR: No template files found!")
    exit()

roi_masks = []
roi_sizes = []

for i, f in enumerate(roi_files):
    roi_img = cv2.imread(f)
    if roi_img is None:
        print(f"ERROR: Cannot load template {f}")
        continue
        
    roi_mask = create_mask(roi_img)
    roi_masks.append(roi_mask)
    roi_sizes.append((roi_mask.shape[1], roi_mask.shape[0]))  # (width, height)
    
    # Debug: แสดงข้อมูล template
    print(f"Template {i}: {f}")
    print(f"  Size: {roi_mask.shape}")
    print(f"  White pixels: {np.sum(roi_mask == 255)}")
    
    # แสดง template mask
    cv2.imshow(f"Template {i} Mask", roi_mask)
    cv2.waitKey(500)

# -------------------------------
# ใช้ cv2.matchTemplate หาตำแหน่งที่เป็นไปได้ทั้งหมด
# -------------------------------
all_detections = []

for roi_idx, (roi_mask, (w, h)) in enumerate(zip(roi_masks, roi_sizes)):
    print(f"\nProcessing template {roi_idx} (size: {w}x{h})")
    
    # ตรวจสอบว่า template ไม่ใหญ่กว่าภาพหลัก
    if w > orig_mask.shape[1] or h > orig_mask.shape[0]:
        print(f"  Template {roi_idx} is larger than main image! Skipping...")
        continue
    
    # Template matching
    result = cv2.matchTemplate(orig_mask, roi_mask, cv2.TM_CCOEFF_NORMED)
    print(f"  Match result shape: {result.shape}")
    print(f"  Max match score: {np.max(result):.3f}")
    print(f"  Min match score: {np.min(result):.3f}")
    
    # ลดเกณฑ์ลงมาก่อนเพื่อ debug
    threshold = 0.3  # ลดลงจาก 0.8 เพื่อดูว่าเจออะไรบ้าง
    locations = np.where(result >= threshold)
    
    print(f"  Found {len(locations[0])} locations above threshold {threshold}")
    
    # เก็บ top 10 detections เพื่อ debug
    if len(locations[0]) > 0:
        # เรียง locations ตาม score
        scores = [result[y, x] for x, y in zip(*locations[::-1])]
        sorted_indices = np.argsort(scores)[::-1]  # เรียงจากสูงไปต่ำ
        
        # เอา top 10 หรือทั้งหมดถ้าน้อยกว่า 10
        top_n = min(10, len(sorted_indices))
        
        for i in range(top_n):
            idx = sorted_indices[i]
            x, y = locations[1][idx], locations[0][idx]  # locations เป็น (y,x)
            score = result[y, x]
            
            all_detections.append({
                'x': x, 'y': y, 'w': w, 'h': h, 
                'score': score, 'roi_idx': roi_idx
            })
            
            print(f"    Detection {i+1}: x={x}, y={y}, score={score:.3f}")

print(f"\nTotal detections collected: {len(all_detections)}")

# -------------------------------
# แสดงผลลัพธ์ทั้งหมดเพื่อ debug
# -------------------------------
if len(all_detections) > 0:
    clone_all = image.copy()
    for det in all_detections:
        # สีต่างกันตาม template
        colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (255, 0, 255)]
        color = colors[det['roi_idx'] % len(colors)]
        
        cv2.rectangle(clone_all, (det['x'], det['y']), 
                      (det['x'] + det['w'], det['y'] + det['h']), color, 2)
        cv2.putText(clone_all, f"T{det['roi_idx']}:{det['score']:.2f}", 
                    (det['x'], det['y']-10), cv2.FONT_HERSHEY_SIMPLEX, 0.4, color, 1)
    
    cv2.imshow("All Detections (Debug)", clone_all)
    print("Press any key to continue...")
    cv2.waitKey(0)
else:
    print("No detections found! Check:")
    print("1. Template files exist and loadable")
    print("2. Template and main image have same color masks") 
    print("3. Template size is reasonable compared to main image")
    print("4. Threshold is not too high")

cv2.destroyAllWindows()

SyntaxError: unterminated string literal (detected at line 18) (2263722713.py, line 18)